# DealSignal Scout (CrewAI)
## Prepared by Husayn El Sharif (Upwork PoC)

This notebook uses a CrewAI multi-agent AI framework (Researcher → Analyst → Writer) to produce a **deal-signal monitoring agent**.

### What it does
- Pulls items from **1–2 external sources** (RSS by default; NewsAPI optional)
- Uses an LLM to **detect deal signals**, tag them, and score relevance
- Stores results in **JSON** 
- Prints a **daily summary** (CLI-style output)

### Notes
- Simple Proof-of-Concept: 1–2 feeds, short prompts, lightweight storage.
- Options to add later: Slack/email alerts, more sources, scheduling.


## 0) Install / Dependencies (if needed)
If you're running this in a fresh environment, install the minimal packages below.

In [29]:
# If needed (uncomment):
# !pip -q install crewai langchain-openai python-dotenv feedparser requests pandas

# Use environment: dlai001


## 1) Imports + Environment Variables
This mirrors your original notebook: load `.env`, configure LLM(s), keep code clean.

In [45]:
import os
import json
import time
import sqlite3
import hashlib
import datetime as dt
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

import feedparser
import requests
from dotenv import load_dotenv, find_dotenv

from langchain_openai import ChatOpenAI
from crewai import Agent, Crew, Task, Process

from pathlib import Path
import json




In [31]:
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

In [32]:
# Load environment variables from .env
load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Add it to your .env file.")


## 2) Config
These are your **configurable criteria** (industries, signal types, score threshold, etc.).
Edit these values to match a client’s needs during a demo.

In [33]:
CONFIG = {
    # Sources: 1–2 external sources are ideal for the POC
    "rss_feeds": [
        "https://techcrunch.com/feed/",
        "https://www.prnewswire.com/rss/news-releases-list.rss",
    ],

    # Optional: NewsAPI (set NEWSAPI_KEY in .env if you want)
    "use_newsapi": False,
    "newsapi_query": "acquisition OR merger OR funding OR strategic alternatives",
    "newsapi_language": "en",
    "newsapi_page_size": 20,

    # Deal criteria (configurable)
    "industries": ["SaaS", "AI", "Healthcare", "Energy", "Industrial"],
    "signal_types": ["M&A", "Funding", "Partnership", "Distress", "Regulatory"],
    "min_relevance_score": 0.70,

    # Runtime / limits
    "max_items_per_source": 15,     # per feed
    "max_total_items": 30,          # optional cap across all sources (helps control cost)

    # Output
    "output_dir": "output",

    # Run label used in filenames
    "run_label": dt.datetime.now().strftime("%Y-%m-%d"),
}



## 3) Utility: Fetch External Data (RSS + optional NewsAPI)
This is the "Monitor" portion of the job scope.

In [34]:
def _hash_id(*parts: str) -> str:
    raw = "||".join([p or "" for p in parts])
    return hashlib.sha256(raw.encode("utf-8")).hexdigest()[:24]

def fetch_rss_items(feeds: List[str], max_items_per_feed: int = 15) -> List[Dict[str, Any]]:
    items: List[Dict[str, Any]] = []
    for feed_url in feeds:
        parsed = feedparser.parse(feed_url)
        for entry in parsed.entries[:max_items_per_feed]:
            title = getattr(entry, "title", "") or ""
            link = getattr(entry, "link", "") or ""
            published = getattr(entry, "published", "") or getattr(entry, "updated", "") or ""
            summary = getattr(entry, "summary", "") or getattr(entry, "description", "") or ""
            item_id = _hash_id(feed_url, title, link)

            items.append({
                "item_id": item_id,
                "source": "rss",
                "source_name": feed_url,
                "title": title,
                "url": link,
                "published": published,
                "content": summary,
                "fetched_at": dt.datetime.utcnow().isoformat(),
            })
    return items

def fetch_newsapi_items(query: str, language: str = "en", page_size: int = 20) -> List[Dict[str, Any]]:
    api_key = os.getenv("NEWSAPI_KEY")
    if not api_key:
        raise ValueError("NEWSAPI_KEY not found. Set it in your .env file to use NewsAPI.")
    url = "https://newsapi.org/v2/everything"
    params = {"q": query, "language": language, "pageSize": page_size, "sortBy": "publishedAt"}
    headers = {"X-Api-Key": api_key}
    r = requests.get(url, params=params, headers=headers, timeout=30)
    r.raise_for_status()
    data = r.json()

    items: List[Dict[str, Any]] = []
    for a in data.get("articles", []):
        title = a.get("title", "") or ""
        link = a.get("url", "") or ""
        published = a.get("publishedAt", "") or ""
        summary = (a.get("description") or "") + "\n" + (a.get("content") or "")
        source_name = (a.get("source") or {}).get("name", "NewsAPI")
        item_id = _hash_id("newsapi", title, link)

        items.append({
            "item_id": item_id,
            "source": "newsapi",
            "source_name": source_name,
            "title": title,
            "url": link,
            "published": published,
            "content": summary.strip(),
            "fetched_at": dt.datetime.utcnow().isoformat(),
        })
    return items


## 4) Storage: JSON
Lightweight, structured, easy to demo. This aligns with the job’s requirement.

In [35]:
# JSON storage helper functions

def save_json(data, filename: str):
    path = OUTPUT_DIR / filename
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    return str(path)

def save_text(text: str, filename: str):
    path = OUTPUT_DIR / filename
    with open(path, "w", encoding="utf-8") as f:
        f.write(text or "")
    return str(path)

## 5) LLM Setup
Keep it minimal: one OpenAI model is enough for this POC (you can swap later).

In [36]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
    openai_api_key=OPENAI_API_KEY,
)


## 6) CrewAI Agents
- Researcher → pulls/filters potential signals
- Analyst → extracts structured fields + scoring
- Writer → produces daily summary output


In [ ]:
researcher_agent = Agent(
    role="Deal Signal Researcher",
    goal=(
        "Identify items from the provided feed batch that may contain business/deal signals "
        "(M&A, funding, partnerships, distress, regulatory)."
    ),
    backstory=(
        "You are a business intelligence researcher. You quickly scan headlines and blurbs "
        "and pick out items that are likely actionable deal signals."
    ),
    llm=llm,
    verbose=False,
    allow_delegation=False,
)

analysis_agent = Agent(
    role="Deal Signal Analyst",
    goal=(
        "For each candidate item, extract structured deal signal fields, assign a relevance score, "
        "and produce short rationale and tags."
    ),
    backstory=(
        "You are a disciplined analyst. You return structured JSON and keep explanations short, "
        "explicit, and decision-ready."
    ),
    llm=llm,
    verbose=False,
    allow_delegation=False,
)

writer_agent = Agent(
    role="Daily Deal Signal Reporter",
    goal=(
        "Write a concise daily summary of the highest scoring signals in a CLI/text-report style "
        "for executives."
    ),
    backstory=(
        "You write crisp bullet summaries: what happened, why it matters, and confidence."
    ),
    llm=llm,
    verbose=False,
    allow_delegation=False,
)


## 7) Tasks
We keep tasks simple and deterministic:
1) Filter down to likely candidates
2) Produce structured JSON for each candidate
3) Produce a daily summary for the run


In [38]:
def build_tasks(feed_batch: List[Dict[str, Any]], config: Dict[str, Any]):
    criteria = {
        "industries": config["industries"],
        "signal_types": config["signal_types"],
        "min_relevance_score": config["min_relevance_score"],
    }

    batch_json = json.dumps(feed_batch, ensure_ascii=False)

    # ---- Task 1: Research (shortlist item_ids) ----
    research_desc = (
        "You will receive a JSON list of news/feed items.\n\n"
        "Return ONLY a JSON list of item_id values that are likely to contain deal signals.\n"
        "Use these signal types: {signal_types}.\n"
        "Industries of interest: {industries}.\n\n"
        "Items JSON:\n"
    ).format(**criteria) + batch_json

    research_task = Task(
        description=research_desc,
        expected_output="A JSON list of item_id strings.",
        agent=researcher_agent,
        human_input=False,
    )

    # ---- Task 2: Analysis (structured extraction) ----
    analysis_desc = (
        "Using the same feed items JSON and the shortlisted item_ids from the Researcher, "
        "analyze each shortlisted item and return ONLY valid JSON (no markdown fences).\n\n"
        "Return a JSON list where each element has:\n"
        "- item_id\n"
        "- signal_type (one of: {signal_types})\n"
        "- relevance_score (0.0 to 1.0)\n"
        "- tags (list of strings)\n"
        "- rationale (1-2 sentences)\n"
        "- llm_summary (2-3 bullets, plain text)\n\n"
        "Scoring guidance:\n"
        "- 0.0-0.4: weak/irrelevant\n"
        "- 0.4-0.7: plausible but not strong\n"
        "- 0.7-1.0: strong actionable signal\n\n"
        "Feed items JSON:\n"
    ).format(**criteria) + batch_json

    analysis_task = Task(
        description=analysis_desc,
        expected_output="A JSON list of structured deal-signal objects.",
        agent=analysis_agent,
        human_input=False,
        context=[research_task],
    )

    # ---- Task 3: Report ----
    report_desc = (
        "Write a CLI-style daily summary for this run.\n\n"
        "STRICT REQUIREMENTS:\n"
        "- Include ONLY items with relevance_score >= {min_relevance_score}\n"
        "- Sort items by relevance_score descending\n"
        "- Use the EXACT title and EXACT url provided in the input data\n"
        "- Do NOT invent, shorten, or replace URLs\n\n"
        "Output format (follow exactly):\n"
        "[YYYY-MM-DD] DealSignal Scout Summary\n"
        "High Confidence Signals: N\n"
        "1) <Title> — <SignalType> (<Score>)\n"
        "   Reason: <1 sentence rationale>\n"
        "   URL: <full URL>\n"
    ).format(**criteria)

    report_task = Task(
        description=report_desc,
        expected_output="A plain-text daily summary report.",
        agent=writer_agent,
        human_input=False,
        context=[analysis_task],
    )

    return research_task, analysis_task, report_task


## 8) Run the POC
This cell performs one on-demand run: fetch → analyze → store → print report.

In [42]:
def run_poc(config: Dict[str, Any]) -> Dict[str, Any]:
    # 1) Fetch
    items = []
    items.extend(fetch_rss_items(config["rss_feeds"], max_items_per_feed=config["max_items_per_source"]))

    if config.get("use_newsapi"):
        items.extend(
            fetch_newsapi_items(
                query=config["newsapi_query"],
                language=config["newsapi_language"],
                page_size=config["newsapi_page_size"],
            )
        )

    # Add run label to each item
    for it in items:
        it["run_label"] = config["run_label"]

    # 2) CrewAI tasks
    research_task, analysis_task, report_task = build_tasks(items, config)

    crew = Crew(
        agents=[researcher_agent, analysis_agent, writer_agent],
        tasks=[research_task, analysis_task, report_task],
        process=Process.sequential,
        verbose=True,
        interactive=False, # no Y/n prompts
        enable_tracing=False, # disable tracing for POC (disables y/n prompt for "would you like to view the trace?")
    )

    _ = crew.kickoff()

    # 3) Parse analysis JSON output (best-effort)
    analysis_raw = getattr(analysis_task.output, "raw", None) if hasattr(analysis_task, "output") else None
    if not analysis_raw:
        analysis_raw = getattr(analysis_task, "output", None)

    analysis_objects = []
    if analysis_raw:
        try:
            analysis_objects = json.loads(str(analysis_raw))
        except Exception:
            # naive fallback: extract outermost JSON list
            s = str(analysis_raw)
            start = s.find("[")
            end = s.rfind("]")
            if start != -1 and end != -1 and end > start:
                try:
                    analysis_objects = json.loads(s[start : end + 1])
                except Exception:
                    analysis_objects = []

    # 4) Normalize + collect structured outputs
    by_id = {it["item_id"]: it for it in items}
    signals = []

    for obj in analysis_objects:
        item_id = obj.get("item_id")
        base = by_id.get(item_id)
        if not base:
            continue

        row = {**base, **obj}

        # Normalize fields defensively
        row["relevance_score"] = float(row.get("relevance_score") or 0.0)

        if isinstance(row.get("tags"), list):
            row["tags"] = row["tags"]
        else:
            row["tags"] = []

        if isinstance(row.get("llm_summary"), list):
            row["llm_summary"] = "\n".join([str(x) for x in row["llm_summary"]])
        else:
            row["llm_summary"] = str(row.get("llm_summary", ""))

        row["rationale"] = str(row.get("rationale", ""))

        signals.append(row)

    # 5) Export artifacts (outside the loop)
    run_label = config["run_label"]

    raw_items_path = save_json(items, f"raw_items_{run_label}.json")
    signals_path = save_json(signals, f"signals_{run_label}.json")

    report_text = getattr(report_task.output, "raw", "")
    summary_path = save_text(report_text, f"summary_{run_label}.txt")

    return {
        "fetched_items": len(items),
        "analyzed_items": len(analysis_objects),
        "signals_exported": len(signals),
        "raw_items_path": raw_items_path,
        "signals_path": signals_path,
        "summary_path": summary_path,
        "report": report_text,
        "run_label": run_label,
    }


In [43]:
out = run_poc(CONFIG)
print(out)
print("\n" + out["report"])


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: adf667b9-88e0-4d7d-96cc-d8c655b32d12                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Deal Signal Researcher                                                                                  │
│                                                                                                                 │
│  Task: You will receive a JSON list of news/feed items.                                                         │
│                                                                                                                 │
│  Return ONLY a JSON list of item_id values that are likely to contain deal signals.                             │
│  Use these signal types: ['M&A', 'Funding', 'Partnership', 'Distress', 'Regulatory'].                           │
│  Industries of interest: ['SaaS', 'AI', 'Healthcare', 'Energy', 'Industrial'].                                  │
│                                                                                                                 │
│  Items JSON:                                                                                                    │
│  [{"item_id": "fbae8afdb8e5551cd9e88155", "source": "rss", "source_name": "https://techcrunch.com/feed/",       │
│  "title": "TikTok agrees to deal to cede control of US business to American investor group", "url":             │
│  "https://techcrunch.com/2025/12/18/tiktok-agrees-to-deal-to-cede-control-of-u-s-business-to-american-investor  │
│  -group/", "published": "Fri, 19 Dec 2025 00:56:44 +0000", "content": "The deal closes a long-running drama in  │
│  which U.S. authorities have sought to compel ByteDance to divest.", "fetched_at":                              │
│  "2025-12-19T01:58:27.422379", "run_label": "2025-12-18"}, {"item_id": "116aca104a828d4c1ee9f95a", "source":    │
│  "rss", "source_name": "https://techcrunch.com/feed/", "title": "Apple becomes a debt collector with its new    │
│  developer agreement", "url":                                                                                   │
│  "https://techcrunch.com/2025/12/18/apple-becomes-a-debt-collector-with-its-new-developer-agreement/",          │
│  "published": "Thu, 18 Dec 2025 23:56:19 +0000", "content": "Apple says it can now collect any money a          │
│  developer owes the tech giant at any time.", "fetched_at": "2025-12-19T01:58:27.422394", "run_label":          │
│  "2025-12-18"}, {"item_id": "1c4920c1442f0ef2b84053ec", "source": "rss", "source_name":                         │
│  "https://techcrunch.com/feed/", "title": "Instacart to pay $60M to settle FTC claims it deceived consumers",   │
│  "url": "https://techcrunch.com/2025/12/18/instacart-to-pay-60m-to-settle-ftc-claims-it-deceived-consumers/",   │
│  "published": "Thu, 18 Dec 2025 21:07:25 +0000", "content": "The federal agency alleged Instacart misled        │
│  consumers with unlawful tactics, causing them to pay higher fees while also denying refunds.", "fetched_at":   │
│  "2025-12-19T01:58:27.422398", "run_label": "2025-12-18"}, {"item_id": "8bf7acb5f763da0ca06c83e6", "source":    │
│  "rss", "source_name": "https://techcrunch.com/feed/", "title": "ChatGPT launches an app store, lets            │
│  developers know it’s open for business", "url":                                                                │
│  "https://techcrunch.com/2025/12/18/chatgpt-launches-an-app-store-lets-developers-know-its-open-for-business/"  │
│  , "published": "Thu, 18 Dec 2025 20:22:50 +0000", "content": "OpenAI is looking to populate its flagship       │
│  chatbot with a host of new user experiences.", "fetched_at": "2025-12-19T01:58:27.422404", "run_label":        │
│  "2025-12-18"}, {"item_id": "bbac8e6235dfb5ca89fc1b91", "source": "rss", "source_name":                         │
│  "https://techcrunch.com/feed/", "title": "Pickle Robot

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Deal Signal Researcher                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ["fbae8afdb8e5551cd9e88155", "1c4920c1442f0ef2b84053ec", "82fcc152779a6b76638d1ef7",                           │
│  "c5e23e7c70666a3556a1ed01", "ad3a956c67bd2d66c9ce87a8", "ac8ab140a29611bfda5989aa",                            │
│  "9f502c5a52e63a6f2841f08a", "05177ce8030f827c17308a22"]                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6e76ae5d-d81b-4f39-b5f3-fa6dc603c7fe                                                                     │
│  Agent: Deal Signal Researcher                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: adf667b9-88e0-4d7d-96cc-d8c655b32d12                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: [2023-10-04] DealSignal Scout Summary                                                            │
│  High Confidence Signals: 5                                                                                     │
│  1) TikTok agrees to divest control of its US operations — M&A (0.9)                                            │
│     Reason: TikTok's agreement to cede control of its US business to an American investor group is a            │
│  significant merger and acquisition signal, indicating a major shift in ownership and regulatory compliance.    │
│     URL: <https://example.com/tiktok-divestment>                                                                │
│                                                                                                                 │
│  2) Kodiak Sciences closed a $184 million public offering — Funding (0.8)                                       │
│     Reason: Kodiak Sciences' successful $184 million public offering indicates strong market confidence and     │
│  funding for its biotechnology initiatives, making it a strong actionable signal.                               │
│     URL: <https://example.com/kodiak-public-offering>                                                           │
│                                                                                                                 │
│  3) Trump Media is merging with TAE Technologies in a $6B deal — M&A (0.8)                                      │
│     Reason: The merger between Trump Media and TAE Technologies represents a strategic expansion into the       │
│  fusion power sector, indicating a significant investment in emerging technologies.                             │
│     URL: <https://example.com/trump-media-merger>                                                               │
│                                                                                                                 │
│  4) Instacart to pay $60M to settle FTC claims — Regulatory (0.7)                                               │
│     Reason: Instacart's $60M settlement with the FTC for deceptive practices highlights regulatory scrutiny in  │
│  the tech industry, which could impact its operational practices and consumer trust.                            │
│     URL: <https://example.com/instacart-ftc-settlement>                                                         │
│                                                                                                                 │
│  5) DXS International confirmed a data breach affecting NHS — Distress (0.7)                                    │
│     Reason: The confirmation of a data breach by a tech provider for NHS England raises concerns about          │
│  cybersecurity and operational integrity in the healthcare sector.                                              │
│     URL: <https://example.com/nhs-data-breach>                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰───────────────────────────────────────────────────────

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 89081668-2711-414d-83a7-f2cfc29fe94c                                                                     │
│  Agent: Daily Deal Signal Reporter                                                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): {'fetched_items': 30, 'analyzed_items': 8, 'signals_exported': 8, 'raw_items_path': 'output/raw_items_2025-12-18.json', 'signals_path': 'output/signals_2025-12-18.json', 'summary_path': 'output/summary_2025-12-18.txt', 'report': "[2023-10-04] DealSignal Scout Summary  \nHigh Confidence Signals: 5  \n1) TikTok agrees to divest control of its US operations — M&A (0.9)  \n   Reason: TikTok's agreement to cede control of its US business to an American investor group is a significant merger and acquisition signal, indicating a major shift in ownership and regulatory compliance.  \n   URL: <https://example.com/tiktok-divestment>  \n\n2) Kodiak Sciences closed a $184 million public offering — Funding (0.8)  \n   Reason: Kodiak Sciences' successful $184 million public offering indicates strong market confidence and funding for its biotechnology initiatives, making it a strong actionable signal.  \n   URL: <https://example.com/k

## 9) Inspect Stored Results
Quickly view what was saved for this run label.

In [46]:
signals_path = out["signals_path"]  # returned from run_poc()

with open(signals_path, "r", encoding="utf-8") as f:
    signals = json.load(f)

print(f"Signals exported for run {out['run_label']}: {len(signals)}")

for r in signals[:5]:
    print("-" * 80)
    print(f"{r['relevance_score']:.2f} | {r['signal_type']} | {r['title']}")
    print(f"URL: {r['url']}")
    print(f"Tags: {r.get('tags', [])}")
    print(f"Rationale: {r.get('rationale', '')}")


Signals exported for run 2025-12-18: 8
--------------------------------------------------------------------------------
0.90 | M&A | TikTok agrees to deal to cede control of US business to American investor group
URL: https://techcrunch.com/2025/12/18/tiktok-agrees-to-deal-to-cede-control-of-u-s-business-to-american-investor-group/
Tags: ['M&A', 'TikTok', 'ByteDance', 'US business']
Rationale: TikTok's agreement to cede control of its US business to an American investor group is a significant merger and acquisition signal, indicating a major shift in ownership and regulatory compliance.
--------------------------------------------------------------------------------
0.70 | Regulatory | Instacart to pay $60M to settle FTC claims it deceived consumers
URL: https://techcrunch.com/2025/12/18/instacart-to-pay-60m-to-settle-ftc-claims-it-deceived-consumers/
Tags: ['Regulatory', 'FTC', 'Instacart', 'settlement']
Rationale: Instacart's $60M settlement with the FTC for deceptive practices highl

## 10) Export Example Output (JSON)
This gives you the “Example output from a real run” deliverable.

In [48]:
# Export signals again (optional manual export)
export_path = OUTPUT_DIR / f"dealsignal_output_{out['run_label']}.json"

with open(export_path, "w", encoding="utf-8") as f:
    json.dump(signals, f, ensure_ascii=False, indent=2)

print("Exported:", str(export_path))



Exported: output/dealsignal_output_2025-12-18.json
